## Convolutions

In [17]:
import torch
from torch import nn

from torch.utils.data import default_collate
from typing import Mapping

from mini_batch_training import*
from data_sets_hf import*
#from miniai.training import *
#from miniai.datasets import *

ModuleNotFoundError: ignored

In [9]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
import pandas as pd,matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor
from urllib.request import urlretrieve

from torch.utils.data import DataLoader
from typing import Mapping

In [10]:
mpl.rcParams['image.cmap'] = 'gray'

In [11]:
MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [12]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])


In [13]:
x_imgs = x_train.view(-1,28,28)
xv_imgs = x_valid.view(-1,28,28)

In [14]:
mpl.rcParams['figure.dpi'] = 30

In [16]:
im3 = x_imgs[7]
show_images(ims)